# Explict recommend

In [18]:
import pymongo_spark
pymongo_spark.activate()
help(pymongo_spark)

Help on module pymongo_spark:

NAME
    pymongo_spark

FILE
    /home/cloudera/mongo-hadoop/spark/src/main/python/pymongo_spark.py

DESCRIPTION
    # Copyright 2015 MongoDB, Inc.
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    # http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.

FUNCTIONS
    BSONFilePairRDD(self, file_path, config=None)
        Create a pair RDD backed by a BSON file.
    
    BSONFileRDD(self, file_path, config=None)
        Create an RDD backed by a BSON file.
    
    activate()
     

In [3]:
import pyspark
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkConf, SparkContext

# Load and parse the data
#sc = pyspark.SparkContext( appName="recommend" ,master="local[*]")
data = sc.textFile("File:/home/cloudera/Desktop/u.data")    #read file at localsite
ratings = data.map(lambda l: l.split('\t'))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# Build the recommendation model using Alternating Least Squares
rank =5
numIterations = 5
model = ALS.train(ratings, rank, numIterations)

# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
model.save(sc, "/home/cloudera/Desktop/myCollaborativeFilter")
sameModel = MatrixFactorizationModel.load(sc, "/home/cloudera/Desktop/myCollaborativeFilter")

Mean Squared Error = 0.638447642646


In [6]:
model.save(sc, "File:/home/cloudera/Desktop/myCollaborativeFilter")

In [19]:
print (predictions.collectAsMap()).keys()[0], (predictions.collectAsMap()).values()[0]
#predictions.saveToMongoDB("mongodb://10.120.28.17:27017/test.webs")

(727, 68) 3.15003492761


In [24]:
dic_list = []
dic_key = predictions.collectAsMap().keys()
dic_val = predictions.collectAsMap().values()
for i in xrange(0,100000):
    userid= dic_key[i][0]
    movid = dic_key[i][1]
    rating = dic_val[i]
    #print userid,movid,rating
    dic = {"userid":userid,"movid":movid,"rating":rating}
    dic_list.append(dic)


# model {"userid":727,"movid":68,"rating":3.15003492761}

In [33]:
import pymongo
from pymongo import MongoClient
client = MongoClient('10.120.28.124', 27017)   #輸入mongodb主機IP與port,連自己主機可以空白,
database = client["test"]                     #[]填入自己設定的db name
collection = database["rdd"]                 #[]填入自己設定的collection name
collection.insert(                       #新增資料進去
        dic_list
    )
client.close()
    

/usr/local/python/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [31]:
import pymongo
from pymongo import MongoClient
client = MongoClient('10.120.28.124', 27017)   #輸入mongodb主機IP與port,連自己主機可以空白,
database = client["test"]                     #[]填入自己設定的db name
collection = database["webs"]

collection.find().count()

1

# Implict  recommend

In [37]:
import pyspark
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkConf, SparkContext

# Load and parse the data
#sc = pyspark.SparkContext( appName="recommend" ,master="local[*]")
data = sc.textFile("File:/home/cloudera/Desktop/u1.data")
ratings = data.map(lambda l: l.split('\t'))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# Build the recommendation model using Alternating Least Squares
rank =5
numIterations = 5
model1 = ALS.trainImplicit(ratings, rank, numIterations,0.1)

# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions1 = model1.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions1)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
model1.save(sc, "File:/home/cloudera/Desktop/u1")
sameModel = MatrixFactorizationModel.load(sc, "File:/home/cloudera/Desktop/u1")

Mean Squared Error = 0.480799177669


Py4JJavaError: An error occurred while calling o1067.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory File:/home/cloudera/Desktop/u1/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:108)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:306)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp

In [46]:
help(model1)

Help on MatrixFactorizationModel in module pyspark.mllib.recommendation object:

class MatrixFactorizationModel(pyspark.mllib.common.JavaModelWrapper, pyspark.mllib.util.JavaSaveable, pyspark.mllib.util.JavaLoader)
 |  A matrix factorisation model trained by regularized alternating
 |  least-squares.
 |  
 |  >>> r1 = (1, 1, 1.0)
 |  >>> r2 = (1, 2, 2.0)
 |  >>> r3 = (2, 1, 2.0)
 |  >>> ratings = sc.parallelize([r1, r2, r3])
 |  >>> model = ALS.trainImplicit(ratings, 1, seed=10)
 |  >>> model.predict(2, 2)
 |  0.4...
 |  
 |  >>> testset = sc.parallelize([(1, 2), (1, 1)])
 |  >>> model = ALS.train(ratings, 2, seed=0)
 |  >>> model.predictAll(testset).collect()
 |  [Rating(user=1, product=1, rating=1.0...), Rating(user=1, product=2, rating=1.9...)]
 |  
 |  >>> model = ALS.train(ratings, 4, seed=10)
 |  >>> model.userFeatures().collect()
 |  [(1, array('d', [...])), (2, array('d', [...]))]
 |  
 |  >>> model.recommendUsers(1, 2)
 |  [Rating(user=2, product=1, rating=1.9...), Rating(user

In [55]:
print model1.recommendProducts(1,5)[0].product

174


In [41]:
print (predictions1.collectAsMap()).keys()[2], (predictions.collectAsMap()).values()[2]
#predictions.saveToMongoDB("mongodb://10.120.28.17:27017/test.webs")

(363, 849) 0.29318396615


In [45]:
dic_list = []
dic_key = predictions1.collectAsMap().keys()
dic_val = predictions1.collectAsMap().values()
with open('test.txt','a+') as ff:
    for i in xrange(0,100000):
        userid= dic_key[i][0]
        movid = dic_key[i][1]
        rating = dic_val[i]
        #print userid,movid,rating
        dic = {"userid":userid,"movid":movid,"rating":rating}
        dic_list.append(dic)
        b = str(userid)+","+str(movid)+","+str(rating)+"\n"
        ff.write(b)
    ff.close()

In [40]:
import pymongo
from pymongo import MongoClient
client = MongoClient('10.120.28.17', 27017)   #輸入mongodb主機IP與port,連自己主機可以空白,
database = client["test"]                     #[]填入自己設定的db name
collection = database["rdd"]                 #[]填入自己設定的collection name
collection.insert(                       #新增資料進去
        dic_list
    )
client.close()

/usr/local/python/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ServerSelectionTimeoutError: 10.120.28.17:27017: timed out